<a href="https://colab.research.google.com/github/jems2019/AlternusVera/blob/socialCredibility/Twitter_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Reading the Data

In [0]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
import nltk.sentiment
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('wordnet')
import matplotlib.pyplot as plt
from scipy import sparse

#helper libraries to take metrics and split data
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.utils.multiclass import unique_labels
from scipy.spatial.distance import cdist


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Read the test, training and valid data from files
# Header = 0 indicates that the first line of the file contains column names,
# As there is no Header, create a column names for each column in the dataset
# delimiter = \t indicates that the fields are seperated by tabs, and 


test_filename = '/content/drive/My Drive/MLSpring2019-Data/AlternusVeraDatasets/input_data/dataset/test.tsv'
train_filename = '/content/drive/My Drive/MLSpring2019-Data/AlternusVeraDatasets/input_data/dataset/train.tsv'
valid_filename = '/content/drive/My Drive/MLSpring2019-Data/AlternusVeraDatasets/input_data/dataset/valid.tsv'

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

train_news = pd.read_csv(train_filename, sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv(test_filename, sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv(valid_filename, sep='\t', names = colnames, error_bad_lines=False)

In [0]:
# Display check the dimensions and the first 2 rows of the file.

print('train dim:',train_news.shape, 'test dim:', test_news.shape)
train_news.iloc[0:2]

train dim: (10240, 14) test dim: (1267, 14)


,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


# Setting up Twitter

In [0]:
# source https://stackabuse.com/accessing-the-twitter-api-with-python/
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  

## CREDENTIALS 1 ##
# credentials['CONSUMER_KEY'] = 'teXgchdxzL9bYnNDT4HctTfUm'
# credentials['CONSUMER_SECRET'] = 'xkNp4LLXdk4vemHF15Bfd0Qn4H6GCHDqY3jTdwheDKdeG0F1kH'
# credentials['ACCESS_TOKEN'] = '837565120435765248-EUarCt01HCRXkCuIHdrWunkJM96FVoD'
# credentials['ACCESS_SECRET'] = 'w8jLAtFxYCeSSVVCTWfKIv9NNYhOvbOBCRyCY13EGKqcK'

## CREDENTIALS 2 ##
credentials['CONSUMER_KEY'] = 'BcPXZRwvvcWCCMKU0tZsN2EGY'
credentials['CONSUMER_SECRET'] = 'QwGSx6nFRYigJsszAGpZemDcgls1kp4otA3mEmsczwVi7u5y5J'
credentials['ACCESS_TOKEN'] = '1050660458254876672-xos6g6ejyVBVvOJVg88VarkIQ8PqF9'
credentials['ACCESS_SECRET'] = '2EPXVunPLsG6CQBBMBXh6AbTpmdC9LWr5MjDIaga2N0xz'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [0]:
!pip install twython
# Import the Twython class
from twython import Twython  

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': 'learn python',  
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }


  Stored in directory: /root/.cache/pip/wheels/c2/b0/a3/5c4b4b87b8c9e4d99f1494a0b471f0134a74e5fb33d426d009
Successfully built twython


In [0]:
!pip install twitter
from twitter import * 

# create Twitter API object
twitter = Twitter(auth = OAuth(creds['ACCESS_TOKEN'],
                  creds['ACCESS_SECRET'],
                  creds['CONSUMER_KEY'],
                  creds['CONSUMER_SECRET']))

     |████████████████████████████████| 61kB 2.9MB/s 


In [0]:
!pip install tweepy
import tweepy
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'],
                  creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'],
                  creds['ACCESS_SECRET'])
api = tweepy.API(auth)

In [0]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.head(5)

,user,date,text,favorite_count
0,CiscoDevNet,Fri May 10 21:34:59 +0000 2019,Ansible is an amazing open source tool for get...,626
1,humble,Thu May 09 22:00:09 +0000 2019,Think Python lets you start with the basics an...,74
2,UAL,Tue May 07 13:15:27 +0000 2019,Coding a Feminist Alexa: Join @UAL_CCI and @fe...,33


In [0]:
# perform user search
results = twitter.users.search(q = '"rob cornilles"')

# loop through each of the users, and print their details
for user in results:
    if user["name"] == 'Rob Cornilles':
        print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]))

@RobCornilles (Rob Cornilles): Oregon's 1st CD


In [0]:
follower_count = api.get_user('RobCornilles').followers_count
print(follower_count)

378


## Speaker Names

In [0]:
# Get unique speaker names

train_news_speaker_unique = pd.DataFrame(train_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
train_news_speaker_unique['speaker'] = train_news_speaker_unique['speaker'].str.replace("-", " ")
train_news_speaker_unique.head(2)

,speaker
0,dwayne bohac
1,scott surovell


In [0]:
# Get unique speaker names

test_news_speaker_unique = pd.DataFrame(test_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
test_news_speaker_unique['speaker'] = test_news_speaker_unique['speaker'].str.replace("-", " ")
test_news_speaker_unique.head(2)

,speaker
0,rick perry
1,katrina shankland


In [0]:
# Get unique speaker names

valid_news_speaker_unique = pd.DataFrame(valid_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
valid_news_speaker_unique['speaker'] = valid_news_speaker_unique['speaker'].str.replace("-", " ")
valid_news_speaker_unique.head(2)

,speaker
0,vicky hartzler
1,chain email


In [0]:
# getting all the unique speakers within all the datasets
speakers = pd.concat([train_news_speaker_unique, test_news_speaker_unique, valid_news_speaker_unique]).drop_duplicates().reset_index(drop=True)
speakers

,speaker
0,dwayne bohac
1,scott surovell
2,barack obama
3,blog posting
4,charlie crist
5,robin vos
6,republican party texas
7,oregon lottery
8,duey stroebel
9,robert menendez


In [0]:
# saving data to csv
speakers.to_csv('speakers_unique.csv')
!cp speakers_unique.csv drive/My\ Drive/

## Twitter Handles

In [0]:
speakers['twitter_handle'] = ''

In [0]:
# Getting twitter handle for speakers

#speakers = speakers[:3]
#train_speaker = train_news

# speaker name
for i,(index,row) in enumerate(speakers.iterrows()):
    if i < 2654: continue # skip first n rows
#for index, row in speakers.iterrows():
  
    name = speakers.at[index,'speaker']
    #name = name.replace("-", " ").lower()
    print(name)
    #perform user search
    res = twitter.users.search(q = name)
    
    # loop through each of the users, and print their details
    for user in res:
        #print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]))
        if user["name"].lower().find(name) == 0:
            print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]) + '\n')
            speakers.at[index,'twitter_handle'] = user["screen_name"]
        break
    #print('\n')

speakers.head()

new georgia encyclopedia
joe theismann
@Theismann7 (Joe Theismann): 

jack murphy
@JackMurphy219 (Jack Murphy): 

emma darnell
kim thatcher
@KimThatcherGOP (Kim Thatcher): Salem Oregon 

atlanta mayor kasim reeds office
gerald mccormick
@GeraldMcCormick (Gerald McCormick): Chattanooga, TN

marshall faulk
@marshallfaulk (Marshall Faulk): EveryWhere

paul levesque
nj can
ryan downton
@Newfino (Ryan Downton): Brampton, ON

steve santarsiero
@SenSantarsiero (Steve Santarsiero): Newtown, PA

joe leibham
@JoeLeibham (Joe Leibham): Sheboygan, WI

national federation blind
jill vogel
@JillHVogel (Jill Vogel): Upperville, Virginia

brian birdwell
american federation children
gerald daugherty
@TeamDaugherty (Gerald Daugherty): Austin, TX

rob eissler
@RobEissler (Rob Eissler): The Woodlands, TX

jim greer
@jwgreer3 (Jim Greer): Dallas, TX

steve kagen
@stevekagen (Steve Kagen, M. D.): Appleton, WI

american international group
ben cannon
yes 82 and 83 campaign
planned parenthood pac
jonathan har

,speaker,twitter_handle
0,dwayne bohac,
1,scott surovell,
2,barack obama,
3,blog posting,
4,charlie crist,


In [0]:
speakers

In [0]:
# saving data to csv
#speakers.to_csv('speakers3.csv')
#!cp speakers3.csv drive/My\ Drive/


## Twitter Followers

In [0]:
twitter = '/content/drive/My Drive/Speakers/speakers_with_twitter.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

#twitter['twitter_handle'] = '@' + twitter['twitter_handle'].astype(str)
twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()

# saving data to csv
#speakers.to_csv('twitter_clean.csv')
#!cp twitter_clean.csv drive/My\ Drive/

,speaker,twitter_handle
0,dwayne bohac,dwayne_bohac
1,scott surovell,ssurovell
2,barack obama,BarackObama
3,charlie crist,CharlieCrist
4,oregon lottery,oregon_lottery


In [0]:
twitter.shape

(1880, 2)

In [0]:
twitter.iloc[1158:]

In [0]:
for i,(index,row) in enumerate(twitter.iterrows()):
    if i < 1157: continue # skip first n rows
#for index, row in twitter.iterrows():
    
    name = twitter.at[index,'twitter_handle'].replace(' \u200f',"")
    #name = name.replace("-", " ").lower()
    print(name)
    follower_count = api.get_user(name).followers_count
    print(" ", follower_count)
    twitter.at[index,'followers'] = follower_count


twitter.head(2)

mitchellvii
  413805
ertharin
  97
Bazag69
  324
lennycurry
  20315
BillGates
  47175167
BenBius
  251
AGBecerra
  63167
charleywilkison
  578
donlemon
  868686
itstimefounder
  841
AmandaFritzRN
  6651
RepHolding
  16253
Dunnie42
  86
peterschweizer
  40070
RepShimkus
  29847
StephanieDesi10
  1
sethillgard
  876
GinaForAustin
  6204
Ahmadinejad1956
  99977
mattgaetz
  118687
janschakowsky
  41094
ChrisVanHollen
  149957
jamesskeff2013
  1
OneNationPAC
  389
McKayKH_95
  142
jamieoliver
  6753729
SteveDoocy
  251765
YvetteMB1
  2776
alroker
  2521563
jim_taricani
  753
BingWest
  1771
RonShipman2
  1
LoriHaasVA
  2023
JohnMorganESQ
  64081
JBPowell11
  293
dougducey
  54077
Schriock1
  22321
hagan2010
  100
RepShankland
  2652
xoladiegaga
  4
DeadlyDoellies
  22
JButing
  102396
fillinator
  6
DaleJr
  2475991
Joe_Sanfelippo
  40385
junior_miller
  176405
MsHelpMeHelpYou
  845
MickMulvaneyOMB
  36361
tedpiccolo
  16
BarbaraLeeSays
  5659
ChefJenCarroll
  38390
pstewart78
  283
penning

,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,NaN
1,scott surovell,ssurovell,NaN


In [0]:
twitter.head(10)

,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,2124.0
1,scott surovell,ssurovell,5969.0
2,barack obama,BarackObama,106128237.0
3,charlie crist,CharlieCrist,32236.0
4,oregon lottery,oregon_lottery,3539.0
5,duey stroebel,SenStroebel,1400.0
6,robert menendez,SenatorMenendez ‏,160365.0
7,bernie s,SenSanders,8332897.0
8,mitt romney,MittRomney,1976486.0
9,doonesbury,DoonesburyComic,2550.0


In [0]:
# saving data to csv
#twitter.to_csv('followers2.csv')
#!cp followers2.csv drive/My\ Drive/

## Number of Tweets

In [0]:
twitter = '/content/drive/My Drive/Speakers/cleaned/twitter_followers.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

#twitter['twitter_handle'] = '@' + twitter['twitter_handle'].astype(str)
twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()


,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,2124
1,scott surovell,ssurovell,5969
2,barack obama,BarackObama,106128237
3,charlie crist,CharlieCrist,32236
4,oregon lottery,oregon_lottery,3539


In [0]:
user = api.get_user('SenSanders')
#print(dir(user))
user.statuses_count
#user.verified

20132

In [0]:
for i,(index,row) in enumerate(twitter.iterrows()):
    if i < 1723: continue # skip first n rows
#for index, row in twitter.iterrows():
    
    name = twitter.at[index,'twitter_handle'].replace(' \u200f',"")
    #name = name.replace("-", " ").lower()
    print(name)
    tweets = api.get_user(name).statuses_count
    verified = api.get_user(name).verified
    print(" ", tweets, " ", verified)
    twitter.at[index,'tweets'] = tweets
    
    if (verified):
      twitter.at[index,'verified'] = 1
    else:
      twitter.at[index,'verified'] = 0
    

    


schuller53rd
  3832   False
BrianWilsonLive
  2172   True
dougainge
  107   False
KampfWarren
  0   False
ThadMcCotter
  10807   True
Mike7Ford
  442   True
Ron4VA
  359   True
RQuig3605
  1433   False
AMBCPSC
  415   True
piersmorgan
  124035   True
KeishaBottoms
  2802   True
SpeakerHastert
  8   False
MickeyAdamsGM
  1605   False
espn
  102083   True
MetroUK
  278106   True
oregoncatalyst
  7368   False
COHealthInst
  6896   False
DeeseOMB
  26   True
BachusAL06
  532   True
representus
  11833   True
Barney_Bishop
  1163   False
juicedageneral
  4449   True
teresa_fedor
  3369   False
EricFehrn
  379   True
MikeNeedham
  722   True
LisaHF1
  1   False
karenohuber
  11995   False
rep89
  5374   True
votersfirstohio
  462   False
AliciaMenendez
  14316   True
psaldana52
  4206   False
breeanabessi
  0   False
OxfamAmerica
  26312   True
lindbeck24
  15   False
john__faulk
  4963   False
David_Prosser
  2662   False
JonathanTurley
  11851   False
jhaas
  10669   False
MarkBellingShow


In [0]:
twitter.head(5)

,speaker,twitter_handle,followers,tweets,verified
0,dwayne bohac,dwayne_bohac,2124,1230.0,0.0
1,scott surovell,ssurovell,5969,8953.0,0.0
2,barack obama,BarackObama,106128237,15625.0,1.0
3,charlie crist,CharlieCrist,32236,2712.0,1.0
4,oregon lottery,oregon_lottery,3539,4240.0,0.0


In [0]:
# saving data to csv
twitter.to_csv('tweets_verified_5.csv')
!cp tweets_verified_5.csv drive/My\ Drive/

## Match Twitter handles with dataset

In [0]:
twitter = '/content/drive/My Drive/Speakers/cleaned/followers_tweets_verified.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

#twitter['twitter_handle'] = '@' + twitter['twitter_handle'].astype(str)
twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()


,speaker,twitter_handle,followers,tweets,verified
0,dwayne bohac,dwayne_bohac,2124,1230,0
1,scott surovell,ssurovell,5969,8953,0
2,barack obama,BarackObama,106128237,15625,1
3,charlie crist,CharlieCrist,32236,2712,1
4,oregon lottery,oregon_lottery,3539,4240,0


In [0]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe


train_news['twitter_handle'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
train_news['followers'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['followers'])
train_news['tweets'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['tweets'])
train_news['verified'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['verified'])

train_news.head(2)

# saving data to csv
train_news.to_csv('train_news_twitter.csv')
!cp train_news_twitter.csv drive/My\ Drive/

In [0]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe


test_news['twitter_handle'] = test_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
test_news.head(2)

# saving data to csv
#test_news.to_csv('test_news_twitter.csv')
#!cp test_news_twitter.csv drive/My\ Drive/

In [0]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe


valid_news['twitter_handle'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
valid_news.head(2)

# saving data to csv
valid_news.to_csv('valid_news_twitter.csv')
!cp valid_news_twitter.csv drive/My\ Drive/